In [147]:
import pandas as pd
from faker import Factory
from faker import Faker
import names
import random
import sys
import math
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import uniform, norm, beta, weibull_min, rv_discrete
from scipy.spatial.distance import cdist, euclidean

%matplotlib inline

In [148]:
fake = Faker()

In [149]:
napa_data_raw = pd.read_excel('HuasData/Napa Building Damage Raw Dataset.xlsx', index_col='Object ID')

In [150]:
napa_data_raw.head()

,Occupancy,Address,Neighborhood,Latitude,Longitude,Original Entry Date,Orignial Placard,Revision Entry Date,Updated Placard,Description
Object ID,,,,,,,,,,
1,RESIDENCE,412 WALNUT,ST. MARE ISLAND,38.101204,-122.274961,2014-08-24,YELLOW,NaT,NaN,CHIMNEY DAMAGE
2,RESIDENCE,416 WALNUT,ST. MARE ISLAND,38.101144,-122.274906,2014-08-24,YELLOW,NaT,NaN,CHIMNEY DAMAGE
3,RESIDENCE,500 WALNUT,ST. MARE ISLAND,38.099772,-122.273636,2014-08-24,YELLOW,NaT,NaN,CHIMNEY DAMAGE
4,RESIDENCE,1035 WALNUT,ST. MARE ISLAND,38.095063,-122.269857,2014-08-24,YELLOW,NaT,NaN,CHIMNEY DAMAGE
5,RESIDENCE,522 WALNUT,ST. MARE ISLAND,38.099569,-122.273438,2014-08-24,YELLOW,NaT,NaN,CHIMNEY DAMAGE


In [151]:
napa_data_hua = pd.read_excel('HuasData/170630 Napa Data.xlsx', index_col='Building Number')

In [152]:
napa_data_hua.head()

,Address,Block #,Red Tags,Tag Color,Occupancy,Area,Year,Age,Value,Permit Issued Date,...,Latitude,Longitude,PH_ENGLISH,PH_NO18UNDER,PASIAN,PHISLAT,PAFRICAN,Income,PHU_OWNED,PHIGHSCH_DIPLOMA
Building Number,,,,,,,,,,,,,,,,,,,,,
5,1266 RIVER PARK BLVD,South,0,Yellow,Residential,1728,1970.0,46.0,721100,2015-01-12,...,38.278174,-122.285368,98.35924,76.338515,5.806452,18.026565,5.16129,42730.0,49.309154,92.684183
6,1360 RIVER PARK BLVD,South,0,Yellow,Residential,1728,1972.0,44.0,648400,NaT,...,38.278132,-122.286463,98.35924,76.338515,5.806452,18.026565,5.16129,42730.0,49.309154,92.684183
8,2261 EVA ST,South,0,Yellow,Residential,1274,1955.0,61.0,465300,2014-09-30,...,38.278660,-122.295215,98.35924,76.338515,5.806452,18.026565,5.16129,42730.0,49.309154,92.684183
9,717 S MINAHEN ST,South,0,Yellow,Residential,1287,1970.0,46.0,442800,NaT,...,38.278143,-122.296806,98.35924,76.338515,5.806452,18.026565,5.16129,42730.0,49.309154,92.684183
10,607 S MINAHEN ST,South,0,Yellow,Residential,1268,1955.0,61.0,439600,2015-05-12,...,38.279145,-122.296324,98.35924,76.338515,5.806452,18.026565,5.16129,42730.0,49.309154,92.684183


In [153]:
napa_data = pd.merge(napa_data_raw, napa_data_hua, on='Address')

napa_data.drop(labels=['Occupancy_x', 'Latitude_x', 'Longitude_x', 'Updated Placard', 'Revision Entry Date', 'Age', 'Foundation', 'Chimney',
                              'Neighborhood', 'Block #', 'Red Tags', 'Orignial Placard', 'PHISLAT', 'Damage Description', 'Description',
                             'PH_ENGLISH', 'PH_NO18UNDER', 'PASIAN', 'PAFRICAN', 'PHU_OWNED', 'PHIGHSCH_DIPLOMA'], 
                      inplace=True, axis=1)

napa_data.columns = ['Address', 'Entry Date', 'Tag',
       'Occupancy', 'Area', 'Year', 'Value', 'Permit Issued',
       'Permit Finaled', 'Damage Level', 'Latitude', 'Longitude', 'Income']

napa_data = napa_data[['Address', 'Occupancy', 'Year', 'Value', 'Area', 'Income', 'Tag',
                                     'Damage Level', 'Entry Date', 'Permit Issued', 
                                     'Permit Finaled', 'Latitude', 'Longitude']]

napa_data = napa_data[~pd.isnull(napa_data['Permit Issued'])]

napa_data.drop_duplicates(subset='Address', inplace=True)

In [154]:
napa_data.head()

,Address,Occupancy,Year,Value,Area,Income,Tag,Damage Level,Entry Date,Permit Issued,Permit Finaled,Latitude,Longitude
2,1006 STONEBRIDGE DR,Residential,1984.0,733400,1503,47946.0,Yellow,2,2014-09-05,2014-09-23,2014-10-21 00:00:00,38.295480,-122.337429
4,1063 STONYBROOK DR,Residential,1974.0,732900,2012,49749.0,Yellow,2,2014-09-05,2014-09-12,2015-03-23 00:00:00,38.305638,-122.345610
10,3696 MONTANA DR,Residential,1978.0,626300,1594,55389.0,Red,4,2014-09-05,2016-04-21,NaN,38.316437,-122.346299
11,3539 LOWREY CT,Residential,NaN,NaN,4500,49749.0,Yellow,2,2014-09-05,2014-09-05,2014-11-10 00:00:00,38.301769,-122.345623
13,25 BLUE JAY CT,Residential,1986.0,758200,1953,49749.0,Yellow,2,2014-09-05,2014-09-25,2014-10-09 00:00:00,38.308398,-122.350411


In [156]:
def set_name(x):
    return fake.name()

def set_income(income_series, count_series):
    x = income_series
    p_x = count_series / count_series.sum()
    income_dist = rv_discrete(name='Income', values=(x, p_x))
    return income_dist.rvs()

def set_savings(income):
    if pd.isnull(income): return
    savings_dist = beta(a=2, b=1, loc=5000, scale = 200000)
    max_pdf = savings_dist.pdf(205000)
    inc_pdf = savings_dist.pdf(income)
    savings_rate = 0.25*(inc_pdf / max_pdf)
    return savings_rate * income

def set_house_value(income):
    if ~pd.isnull(income): return
    min_house_price_multiplier = 2
    return income*uniform.rvs(loc = min_house_price_multiplier, scale = 1)

def set_house_area(value):
    if pd.isnum(value): return
    dollar_per_sf = 150
    if int(value / dollar_per_sf) < 500:
        return 500
    else:
        return value / dollar_per_sf

def set_mortgage_payment(value):
    if pd.isnull(value): return
    monthly_rate = 0.05/12
    num_payments = 30*12
    down_payment = 0.1
    loan_value = value - value * down_payment
    return -np.pmt(monthly_rate,num_payments, loan_value)

def set_occupancy(income):
    if income >= 50000:
        return 'Single Family Dwelling'
    elif uniform.rvs(0,1) >= 0.2:
        return 'Single Family Dwelling'
    else:
        return 'Mobile Home'

def set_listing():
    if uniform.rvs(0,1) >= 0.3333:
        return False
    else:
        return True

def set_credit(x):
    return int(uniform.rvs(300, 350))

def set_insurance(income):
    insurance_dist = beta(a=2, b=1, loc=5000, scale = 200000)
    max_pdf = insurance_dist.pdf(205000)
    inc_pdf = insurance_dist.pdf(income)
    if inc_pdf / max_pdf > 0.5:
        return 0.8
    else:
        return 0.0

def set_bedrooms(area):
    if area <= 500:
        return 0
    else:
        bedrooms_pct = 0.3
        avg_sf = 200
        return int((bedrooms_pct * area) / avg_sf ) 

def set_bathrooms(area):
    if area <= 500:
        return 1
    else:
        bathrooms_pct = 0.1
        avg_sf = 100
        return max(int((bathrooms_pct * area) / avg_sf ), 1)

In [157]:
napa_data['Name'] = np.nan
napa_data['Credit'] = np.nan
napa_data['Savings'] = napa_data['Income']
napa_data['Bedrooms'] = napa_data['Area']
napa_data['Bathrooms'] = napa_data['Area']
napa_data['Mortgage Payment'] = napa_data['Value']
napa_data['Listing'] = np.nan
napa_data['Insurance'] = napa_data['Income']


In [144]:
napa_data.loc[napa_data['Value'].isnull(),'Value'] = napa_data['Income'].apply(set_house_value)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [160]:
napa_data['Value']

2        733400
4        732900
10       626300
11          NaN
13       758200
19       482600
27       511900
28       491100
34       471900
36       420600
37          NaN
42       425000
43       480300
45       488000
46       514600
48       426900
49       407000
55       574100
61       471800
62       403100
67       688700
69       491900
71       468400
72       446900
73      1025700
74       800400
75       758200
76       759900
79       504800
81       586700
         ...   
1320     504000
1321        NaN
1323     719100
1325        NaN
1327     696400
1328     726600
1332     476200
1333     557900
1335     828100
1337     592900
1338     583700
1339     584000
1341     592600
1343     703300
1345     484500
1346     531500
1348     629000
1350     737000
1353     500300
1357     444800
1359     519500
1361     518500
1362     444000
1366     416500
1367     434200
1370     535600
1371     470200
1372     650400
1374     601300
1375     749000
Name: Value, Length: 669

In [167]:
napa_data[napa_data['Value']=='Year']

,Address,Occupancy,Year,Value,Area,Income,Tag,Damage Level,Entry Date,Permit Issued,...,Latitude,Longitude,Name,Credit,Savings,Bedrooms,Bathrooms,Mortgage Payment,Listing,Insurance


In [166]:
max(napa_data[~pd.isnull(napa_data['Value'])])

'Year'

In [83]:
napa_data['Credit'].apply(set_credit);
napa_data['Name'].apply(set_name);
napa_data['Savings'].apply(set_savings);

In [ ]:
napa_data.loc[napa_data['Value'].isnull(),'Value']

In [95]:
napa_data['Mortgage Payment'].apply(set_mortgage_payment)

2       None
4       None
10      None
11      None
13      None
19      None
27      None
28      None
34      None
36      None
37      None
42      None
43      None
45      None
46      None
48      None
49      None
55      None
61      None
62      None
67      None
69      None
71      None
72      None
73      None
74      None
75      None
76      None
79      None
81      None
        ... 
1320    None
1321    None
1323    None
1325    None
1327    None
1328    None
1332    None
1333    None
1335    None
1337    None
1338    None
1339    None
1341    None
1343    None
1345    None
1346    None
1348    None
1350    None
1353    None
1357    None
1359    None
1361    None
1362    None
1366    None
1367    None
1370    None
1371    None
1372    None
1374    None
1375    None
Name: Mortgage Payment, Length: 669, dtype: object